In [1]:
import json
import networkx as nx
import numpy as np
import math
from numpy.linalg import norm

In [2]:
def showGraph(G):
    pos = {}
    for node in G.nodes(data = True):
        pos[node[0]] = [int(p) for p in node[1]['position'][0:2]]
    nx.draw(G, pos = pos)


In [3]:
def createGraph(graph):
    G = nx.Graph()
    for node in graph['node']:
        G.add_node(node['storeName'], 
            storeName = node['storeName'], 
            position = (node['posX'], node['posY']))
    # for edge in graph['edge']:
        # x1,y1,z1 = map(int,G.nodes[edge['node'][0]]['position'])
        # x2,y2,z2 = map(int,G.nodes[edge['node'][1]]['position'])
        # G.add_edge(edge['node'][0], edge['node'][1], weight = int(abs((x1-x2) +(y1-y2) +(z1-z2))))
    return G

In [4]:
def alignNodes(path):
    G = []
    f = open(path)
    graph = json.load(f, parse_int=str)
    for node in graph['storeNodes']:
        G.append([node['storeName'],int(node['posX']),int(node['posY'])])
    for i in range(len(G)):
        for j in range(i+1, len(G)):
            for k in range(1,3):
                if(1 <= abs(G[i][k]-G[j][k]) < 2):
                    G[i][k] = G[j][k] = (G[i][k]+G[j][k])/2
    for i in range(len(G)):
        for j in range(i+1, len(G)):
            for k in range(1,3):
                if(abs(G[i][k]-G[j][k]) < 2):
                    G[i][k] = G[j][k] = int((G[i][k]+G[j][k])//2)
                    # print(G[i],G[j]) 
    return G

In [5]:
def getCorridors(G):
    G = sorted(G,key=lambda x: x[1])
    Corners = [] 
    for i in range(len(G)):
        for j in range(i+1, len(G)):
            if(not(G[i][1] == G[j][1] or G[i][2] == G[j][2])):
                if(not ([G[i][1],G[j][2]] in Corners)):
                    Corners.append([G[i][1],G[j][2]])
                if(not([G[j][1],G[i][2]] in Corners)):
                    Corners.append([G[j][1],G[i][2]])
    center = [0,0]
    for i in Corners:
        center[0] += i[0]
        center[1] += i[1]

    center = [int(center[0]/5),int(center[1]/5)]
    try:
        Corners.pop(Corners.index(center))
    except:
        pass
    Corridors = []
    for i in range(len(Corners)):
            for j in range(i+1,len(Corners)):
                if(Corners[i][0]==Corners[j][0]):
                    Corridors.append([0,np.array(Corners[i]),np.array(Corners[j])])
                if(Corners[i][1]==Corners[j][1]):
                    Corridors.append([1,np.array(Corners[i]),np.array(Corners[j])])
    return Corridors

In [6]:
# d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)
def getCorridorNodes(storeNodes,Corridors):
    CorridorNodes = []
    for storeNode in storeNodes:
        distance = [9999]
        # print(storeNode)
        newNode = np.copy(storeNode)            
        for [axis,p1,p2] in Corridors:
            d = norm(np.cross(p2-p1, p1-storeNode))/norm(p2-p1)
            if(distance[0] > d):
                distance = [d,axis,p1]
            if(d == 0):
                if(norm(p1-storeNode)>norm(p2-storeNode)):
                    newNode = np.copy(p2)
                else:
                    newNode = np.copy(p1)
                break
        newNode[distance[1]] = distance[2][distance[1]]
        CorridorNodes.append(newNode)
        # print(distance[0],storeNode,newNode)
    return CorridorNodes

In [ ]:
def connectCorridorNodes(corridorNodes):
    

In [7]:
alignedStoreNodes = alignNodes('./Jsons/S3.json')       
StoreNode = np.array([[int(X[1]),int(X[2])] for X in alignedStoreNodes ])
print(StoreNode)

[[ 5  2]
 [ 9  2]
 [16  4]
 [16 11]
 [ 9 13]
 [ 7  8]
 [12  8]
 [ 3  8]]


In [8]:
alignedCorridorNodes = alignNodes('./Jsons/C3.json')
print(alignedCorridorNodes)
corridorNodes = np.array([[int(X[1]),int(X[2])] for X in alignedCorridorNodes ])
corridorCorners = getCorridors(alignedCorridorNodes)
# print(corridorCorners)

[['c1', 5, 8], ['c2', 10, 11], ['c3', 14, 8], ['c4', 10, 5]]


In [9]:
corridorNodes = getCorridorNodes(StoreNode,corridorCorners)
print(corridorNodes)

[array([5, 5]), array([9, 5]), array([16,  5]), array([14, 11]), array([ 9, 11]), array([5, 8]), array([14,  8]), array([5, 8])]
